In [1]:
#step one, get the top 100
import pandas as pd
from bs4 import BeautifulSoup
import requests

import os
# shazamapi needds ffmpeg , but it doesn't really have to be imported anywhere : import ffmpeg


In [2]:
#Initialize SpotiPy with user credentias
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config



In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))
  

In [4]:
# local libraries
import shazaam

In [5]:

shazaam.shazaam(debug=True)


recording for 5 seconds
Starting: Play music now!
finished
that was "Enemy (From the series "Arcane League of Legends")" from Imagine Dragons, JID & League of Legends


('Enemy (From the series "Arcane League of Legends")',
 'Imagine Dragons, JID & League of Legends')

In [6]:
def soundex(name):


    """
    The Soundex algorithm assigns a 1-letter + 3-digit code to strings,
    the intention being that strings pronounced the same but spelled
    differently have identical encodings; words pronounced similarly
    should have similar encodings.
    """

    soundexcoding = [' ', ' ', ' ', ' ']
    soundexcodingindex = 1

    #           ABCDEFGHIJKLMNOPQRSTUVWXYZ
    mappings = "01230120022455012623010202"

    soundexcoding[0] = name[0].upper()

    for i in range(1, len(name)):

         c = ord(name[i].upper()) - 65

         if c >= 0 and c <= 25:

             if mappings[c] != '0':

                 if mappings[c] != soundexcoding[soundexcodingindex-1]:

                     soundexcoding[soundexcodingindex] = mappings[c]
                     soundexcodingindex += 1

                 if soundexcodingindex > 3:

                     break

    if soundexcodingindex <= 3:
        while(soundexcodingindex <= 3):
            soundexcoding[soundexcodingindex] = '0'
            soundexcodingindex += 1

    return ''.join(soundexcoding)

In [7]:
def is_recognizable():
    return len(shazam_info[1]["matches"])>0

def get_song_info():
    return shazam_info[1]["track"]["title"], shazam_info[1]["track"]["subtitle"]

In [8]:
URL = 'https://www.billboard.com/charts/hot-100/'
Top100Website = requests.get(URL)
result = BeautifulSoup(Top100Website.content, 'html.parser')

In [9]:
result.find("span", id="count_texttitle-of-a-story")

In [10]:
line_number = 0
place_number = 0
Song = "";
Artists = "";
Top_100 = pd.DataFrame(columns=['place','song','artists', 'song_soundex'])
print(str(Top_100))

for lines in result.find_all("li", {"o-chart-results-list__item"}):
    line_number += 1

 
    if((line_number-4)%14==0):
        place_number += 1
        internal_line = 1
        for current_line in lines.get_text().splitlines():

            if(current_line.strip()!=""):
                if(internal_line==1):
                    Song = current_line.strip()
                else:
                    Artists = current_line.strip()

                    Top_100.loc[place_number -1] = [place_number, Song, Artists, soundex(str(Song))]
 
                internal_line += 1

Empty DataFrame
Columns: [place, song, artists, song_soundex]
Index: []


In [11]:
#temporary
Top_100

,place,song,artists,song_soundex
0,1,As It Was,Harry Styles,A232
1,2,First Class,Jack Harlow,F623
2,3,Heat Waves,Glass Animals,H312
3,4,Big Energy,Latto,B256
4,5,Enemy,Imagine Dragons X JID,E500
...,...,...,...,...
95,96,P Power,Gunna Featuring Drake,P160
96,97,Money So Big,Yeat,M521
97,98,Blick Blick!,Coi Leray & Nicki Minaj,B421
98,99,Fall In Love,Bailey Zimmerman,F454


In [12]:
import config
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,  client_secret= config.client_secret))


In [31]:
recommend_me_something_for_this = input("Please enter a song to base a recommendation on ")

Please enter a song to base a recommendation on shazaam


In [35]:
song_selector_dictionary = {}
song_selected = ""
if(recommend_me_something_for_this=="shazaam"):
    #use the local shazaam library to identify the song currently playing
    recorded = shazaam.shazaam()

    #returns false if nothing was found
    if(recorded != False):
        #and a tuple of song and artist if it worked
        recorded_song, recorded_artist = recorded 
        print(recorded_song + " " + recorded_artist)
    
        #replace the input with the actual song
        recommend_me_something_for_this = recorded_song
  
Number_Of_Songs_Found = len(Top_100[Top_100['song_soundex'] == soundex(recommend_me_something_for_this.strip())])
if(Number_Of_Songs_Found>0):
    #find a new song
    print("Let me recommend you:")
    recommendation_dataframe = Top_100.sample(n=1)
    print(str(recommendation_dataframe ))
else:
    if(len(recommend_me_something_for_this.strip())>0):
        search_results = sp.search(q=recommend_me_something_for_this.strip(),limit=50,market="GB")
        if(len(search_results["tracks"])>1):
            print("Please select a number regarding the many titles of this name")
            for i in range(0,len(search_results["tracks"]["items"])):

                print("["+str(i+1)+"] " + str(search_results["tracks"]["items"][i]["name"]))
                song_selector_dictionary[str(i+1)] = str(search_results["tracks"]["items"][i]["id"])
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][i]["artists"])):

                    print(str(search_results["tracks"]["items"][i]["artists"][j]["name"]))
                print(" ")
           # print(str(search_results["tracks"]["items"][i]["name"]))        
        else:
            if(len(search_results)==1):
                print(str(search_results["tracks"]["items"][0]["name"]))
                print("by")
                for j in range(0,len(search_results["tracks"]["items"][0]["artists"])):
                    print("  " + str(search_results["tracks"]["items"][0]["artists"][j]["name"]))

        song_selected = input("which number do you choose? ")
    # print("“Unfortunately, the song is not in the hot list!")    

Please select a number regarding the many titles of this name
[1] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[2] Cantaloop (Flip Fantasia) - Goldfish Remix
by
Us3
 
[3] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[4] Cantaloop (Flip Fantasia) [Goldfish Remix]
by
GoldFish
Us3
 
[5] Cantaloop (Flip Fantasia) (Re-Recorded / Remastered)
by
Us3
 
[6] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[7] Cantaloop (Flip Fantasia) - Instrumental
by
Us3
 
[8] Cantaloop - Nellee Hooper Remix
by
Us3
 
[9] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[10] Cantaloop (Flip Fantasia) - Rerecorded
by
Us3
 
[11] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[12] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[13] Cantaloop (Flip Fantasia)
by
Us3
Rahsaan
Gerard Presencer
 
[14] Cantaloop (Flip Fantasia) - Us3 Smokin’ Mix
by
Us3
 
[15] Cantaloop (Flip Fantasia) - Us3 House Remix
by
Us3
 
[16] Cantaloop (Flip Fantasia)
b

In [36]:
track_id = song_selector_dictionary[song_selected]



In [37]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

